In [ ]:
import pandas as pd
import datetime
pd.set_option('display.max_columns', None)
from pyexcelerate import Workbook
from decimal import Decimal
import numpy as np
import time
import re
from tqdm import tqdm
import warnings

warnings.filterwarnings('ignore')

def rev_to_dict(ind,find):
#     ind = ind.astype('str')
    df = pd.DataFrame(index=ind,data=find.values,columns=[find.name])
    df = df.groupby(df.index).first()
    dictt =  df.to_dict('index')
    return dictt

def header_parser(df):

    lst = list(df.columns)

    value = 0
    for index,col in enumerate(lst):
        if col is np.nan:
            lst[index] = lst[index-1]+str(value)
            value +=1
        else:
            value  = 0

    a,b = [],[]
    for col in lst:
        a.append(col)
        b.append(lst.count(col))

    a = pd.Series(a)
    b = pd.Series(b)
    b.name = 'val'    
    s  = rev_to_dict(a,b)

    ls_k = list(s.keys())

    for col in ls_k:
        if s[col]['val']>1:
            for ind in range(0,s[col]['val']):
                for ind_,col_ in enumerate(lst):
                    if col_==col:
                        lst[ind_] = col_+str(ind)
                        break

    df.columns = lst
    
    return df



def save_pyex(path,df):
    wb = Workbook()
    wb.new_sheet('sheet1',data =  [df.columns.tolist(),]+df.values.tolist())
    wb.save(path)

    
import copy
import csv
import dask
from dask import dataframe as dd 
from multiprocessing.pool import ThreadPool
dask.config.set(pool=ThreadPool(6))

def get_arr_from_csv(dir_and_filename):
    arr = []
    with open(dir_and_filename) as File:
        reader = csv.reader(File, delimiter=';', quotechar=',',
                            quoting=csv.QUOTE_MINIMAL)
        for row in reader:
            arr.append(row)
    
    return arr

def get_dask_arr(directory,file_data_name,file_meta_name):
    arr_mtd = get_arr_from_csv(directory+file_meta_name+'.CSV')
    arr_data = get_arr_from_csv(directory+file_data_name+'.CSV')
    dict_meta = dict((x[0].strip(),x[1].strip()) for x in arr_mtd if len(x)>3 and x[0]!='COLUMN')
    dict_meta_types = dict((x[0].strip(),x[3].strip()) for x in arr_mtd if len(x)>3 and x[0]!='COLUMN')
    dfr = pd.DataFrame(data = arr_data,columns = list(dict_meta.values()))
    
    for ind,col in enumerate(dfr.columns,1):
        if types_pnds[dict_meta_types[str(ind)]]=='float64':
#             dfr[col] = dfr[col].apply(lambda x:str(x).replace(regex = r'^-$',new = ''))
#             dfr[col] = dfr[col].replace(regex=r'-$', value='')
            dfr[col] = dfr[col].apply(lambda x:x[-1]+x[:-1] if len(re.findall(r'-$',x))>0 else x)
        dfr[col] = dfr[col].astype(types_pnds[dict_meta_types[str(ind)]])
    
#     sd = dd.from_pandas(dfr, npartitions=100)
    sd = dfr
    return sd

def get_df_gr_pandas(df,index_):
    
    index_join_group = copy.copy(index_) 

    vl06_index_join_group_start = copy.copy(index_join_group)
    vl06_index_join_group_start.append('ZKFGDOCIN')



    print(index_join_group)
    
    df['gr_index']=''
    for ind, col in enumerate(index_join_group):
        if ind==len(index_join_group)-1:
            df['gr_index'] = df['gr_index'].astype('str')+df[col].astype('str')
        else:
            df['gr_index'] = df['gr_index'].astype('str')+df[col].astype('str')+'----'

    
    
    potr = df[['gr_index','/BIC/ZMM1Q01']].groupby('gr_index').sum().reset_index()
    potr =  potr.rename(columns={"/BIC/ZMM1Q01":"_Потребность,кол-во"})
    potr = potr.set_index('gr_index')
    
    
    potr_usd = df[['gr_index','/BIC/ZMM1A01']].groupby('gr_index').sum().reset_index()
    potr_usd =  potr_usd.rename(columns={"/BIC/ZMM1A01":"_Потребность,стоимость,USD"})
    potr_usd = potr_usd.set_index('gr_index')
    
    ost_raspr = df[['gr_index','/BIC/ZMM1Q22']].groupby('gr_index').sum().reset_index()
    ost_raspr =  ost_raspr.rename(columns={"/BIC/ZMM1Q22":"Распределение остатка,кол-во"})
    ost_raspr = ost_raspr.set_index('gr_index')
    
    
    ost_raspr_KZT = df[['gr_index','VALST_KZT']].groupby('gr_index').sum().reset_index()
    ost_raspr_KZT =  ost_raspr_KZT.rename(columns={"VALST_KZT":"Сумма распределенного остатка,KZT"})
    ost_raspr_KZT = ost_raspr_KZT.set_index('gr_index')
    
    ost_raspr_USD = df[['gr_index','VALST_USD']].groupby('gr_index').sum().reset_index()
    ost_raspr_USD =  ost_raspr_USD.rename(columns={"VALST_USD":"Сумма распределенного остатка,USD"})
    ost_raspr_USD = ost_raspr_USD.set_index('gr_index')
    
    
    vl_06 = df[df['DOCTYPE'].isin(['VL06'])][['gr_index','ZKFGDOCIN']].groupby('gr_index').sum().reset_index()
    vl_06 =  vl_06.rename(columns={"ZKFGDOCIN":"Товар в пути(VL06),кол-во"})
    vl_06 = vl_06.set_index('gr_index')
    
    znb_5 = df[df['DOCTYPE'].isin(['ZNB5'])][['gr_index','ZKFGDOCIN']].groupby('gr_index').sum().reset_index()
    znb_5 =  znb_5.rename(columns={"ZKFGDOCIN":"Товар в пути(ZNB5),кол-во"})
    znb_5 = znb_5.set_index('gr_index')
    
    
    znb_2_3_4 = df[df['DOCTYPE'].isin(['ZNB2','ZNB3','ZNB4'])][['gr_index','ZKFGDOCIN']].groupby('gr_index').sum().reset_index()
    znb_2_3_4 =  znb_2_3_4.rename(columns={"ZKFGDOCIN":"Законтрактовано(ZNB2-4-ПртПзк),кол-во"})
    znb_2_3_4 = znb_2_3_4.set_index('gr_index')
    
    
    nb = ar[ar['DOCTYPE'].apply(lambda x: len(re.findall('^NB\d\d',str(x)))>0 )][['gr_index','ZKFGDOCIN']].groupby('gr_index').sum().reset_index()
    nb =  nb.rename(columns={"ZKFGDOCIN":"Заявка на закуп(NB),кол-во"})
    nb = nb.set_index('gr_index')
    
    r_rash = df[['gr_index','ZKFGOUT']].groupby('gr_index').sum().reset_index()
    r_rash =  r_rash.rename(columns={"ZKFGOUT":"Распределение расхода, кол-во"})
    r_rash = r_rash.set_index('gr_index')
    
    
    r_prih_1 = df[df['/BIC/ZDECADE'].isin(['1'])][['gr_index','ZKFGIN']].groupby('gr_index').sum().reset_index()
    r_prih_1 =  r_prih_1.rename(columns={"ZKFGIN":"Распределение прихода 1декада, кол-во"})
    r_prih_1 = r_prih_1.set_index('gr_index')
    
    r_prih_2 = df[df['/BIC/ZDECADE'].isin(['2'])][['gr_index','ZKFGIN']].groupby('gr_index').sum().reset_index()
    r_prih_2 =  r_prih_2.rename(columns={"ZKFGIN":"Распределение прихода 2декада, кол-во"})
    r_prih_2 = r_prih_2.set_index('gr_index')
    
    r_prih_3 = df[df['/BIC/ZDECADE'].isin(['3'])][['gr_index','ZKFGIN']].groupby('gr_index').sum().reset_index()
    r_prih_3 =  r_prih_3.rename(columns={"ZKFGIN":"Распределение прихода 3декада, кол-во"})
    r_prih_3 = r_prih_3.set_index('gr_index')
    
    
    r_prih_sum = df[df['FLAG_DISTR'].isin(['I','N','P','T'])][['gr_index','/BIC/ZCPPVCC']].groupby('gr_index').sum().reset_index()
    r_prih_sum =  r_prih_sum.rename(columns={"/BIC/ZCPPVCC":"Сумма распределенного прихода, USD"})
    r_prih_sum = r_prih_sum.set_index('gr_index')
    
    r_rash_sum = df[df['FLAG_DISTR'].isin(['U','O'])][['gr_index','/BIC/ZCPPVCC']].groupby('gr_index').sum().reset_index()
    r_rash_sum =  r_rash_sum.rename(columns={"/BIC/ZCPPVCC":"Сумма распределенного расхода, USD"})
    r_rash_sum = r_rash_sum.set_index('gr_index')
    
    
    
    
    
    
    dfr = pd.concat([potr,potr_usd,ost_raspr,ost_raspr_KZT,ost_raspr_USD,vl_06,znb_5,znb_2_3_4,nb,r_rash,r_rash_sum,r_prih_1,r_prih_2,r_prih_3,r_prih_sum],axis = 1)
    
    dfr = dfr.reset_index()
    
    for ind,col in enumerate(index_join_group):
        dfr[col] = dfr['index'].apply(lambda x: str(x).split('----')[ind])
    dfr = dfr.drop(['index'],axis = 1)
    
    for col in dfr.columns:
        if dfr[col].dtype=='float64':
            dfr[col].fillna(0,inplace = True)
    
    
    return dfr

In [ ]:
directory = 'C:\\Users\\NKedrun\\AppData\\Local\\VirtualStore\\'
types_pnds = {'CHAR':'category','CUKY':'category','CURR':'float64','FLTP':'float64','NUMC':'category','QUAN':'float64','UNIT':'category','DATS':'category'}

In [ ]:
ar = get_dask_arr(directory,'ZPR1OH01','S_ZPR1OH01')


In [ ]:
tt1  = get_df_gr_pandas(df = ar,index_ = ['COMP_CODE','/BIC/ZNPLANT','CALDAY','/BIC/ZRMTRLEAD'])

In [ ]:
tt1

In [ ]:
# for ind, i in enumerate(np.array_split(dfr,3)) :
save_pyex(df=tt1,path=directory+'open_hub_.xlsx')

In [ ]:
tt1

In [ ]:
ar

In [ ]:
potr = ar[['gr_index','/BIC/ZMM1Q01','CALYEAR']].pivot_table(index=['gr_index'],aggfunc='sum',columns=['CALYEAR'],values = ['/BIC/ZMM1Q01'],fill_value=0,margins_name='tgtrtgr',margins = True)
# potr =  potr.rename(columns={"/BIC/ZMM1Q01":"_Потребность,кол-во"})
# potr = potr.set_index('gr_index')

In [ ]:
potr

In [ ]:
potr_usd = ar[['gr_index','/BIC/ZMM1A01','CALYEAR','/BIC/ZMODUL']].pivot_table(index=['gr_index'],aggfunc='sum',columns=['CALYEAR','/BIC/ZMODUL'],values = ['/BIC/ZMM1A01'],fill_value=0,margins_name='tgtrtgr',margins = True)
#     potr_usd =  potr_usd.rename(columns={"/BIC/ZMM1A01":"_Потребность,стоимость,USD"})
#     potr_usd = potr_usd.set_index('gr_index')

In [ ]:
[('ttt',x[1]) for x in potr_usd.columns]

In [ ]:
potr_usd.columns = pd.MultiIndex.from_tuples([('ttt',x[1]) for x in potr_usd.columns])

In [ ]:
potr_usd

In [ ]:
pd.concat([potr,potr_usd],axis = 1)

In [ ]:
def get_multiindex(cols,ins_value):
    arr = []
    if len(cols)>1:
        for el in cols:
            a = list(el)
            a[0] = ins_value
            a = tuple(a)
            arr.append(a)
        return pd.MultiIndex.from_tuples(arr)
    if len(cols)==1:
        return [ins_value]

def get_df_pvt_pandas(df,index_,columns_):
    
    index_join_group = copy.copy(index_) 

    vl06_index_join_group_start = copy.copy(index_join_group)
    vl06_index_join_group_start.append('ZKFGDOCIN')



    print(index_join_group)
    
    df['gr_index']=''
    for ind, col in enumerate(index_join_group):
        if ind==len(index_join_group)-1:
            df['gr_index'] = df['gr_index'].astype('str')+df[col].astype('str')
        else:
            df['gr_index'] = df['gr_index'].astype('str')+df[col].astype('str')+'----'

    
    
    gr = ['gr_index','/BIC/ZMM1Q01']
    if columns_!=None:
        gr.extend(columns_)
    potr = df[gr].pivot_table(index=['gr_index'],aggfunc='sum',columns=columns_,values = ['/BIC/ZMM1Q01'],fill_value=0,margins_name='_Потребность,кол-во',margins = True)
    potr.columns = get_multiindex(potr.columns,'_Потребность,кол-во')
    potr = potr[:-1]
    
    
    gr = ['gr_index','/BIC/ZMM1A01']
    if columns_!=None:
        gr.extend(columns_)
    potr_usd = df[gr].pivot_table(index=['gr_index'],aggfunc='sum',columns=columns_,values = ['/BIC/ZMM1A01'],fill_value=0,margins_name='_Потребность,стоимость,USD',margins = True)
    potr_usd.columns = get_multiindex(potr_usd.columns,'_Потребность,стоимость,USD')
    potr_usd = potr_usd[:-1]
    
    
    gr = ['gr_index','/BIC/ZMM1Q22']
    if columns_!=None:
        gr.extend(columns_)
    ost_raspr = df[gr].pivot_table(index=['gr_index'],aggfunc='sum',columns=columns_,values = ['/BIC/ZMM1Q22'],fill_value=0,margins_name='Распределение остатка,кол-во',margins = True)
    ost_raspr.columns = get_multiindex(ost_raspr.columns,'Распределение остатка,кол-во')
    ost_raspr = ost_raspr[:-1]
    
    
    gr = ['gr_index','VALST_KZT']
    if columns_!=None:
        gr.extend(columns_)
    ost_raspr_KZT = df[gr].pivot_table(index=['gr_index'],aggfunc='sum',columns=columns_,values = ['VALST_KZT'],fill_value=0,margins_name='Сумма распределенного остатка,KZT',margins = True)
    ost_raspr_KZT.columns = get_multiindex(ost_raspr_KZT.columns,'Сумма распределенного остатка,KZT')
    ost_raspr_KZT = ost_raspr_KZT[:-1]
    
    gr = ['gr_index','VALST_USD']
    if columns_!=None:
        gr.extend(columns_)
    ost_raspr_USD = df[gr].pivot_table(index=['gr_index'],aggfunc='sum',columns=columns_,values = ['VALST_USD'],fill_value=0,margins_name='Сумма распределенного остатка,USD',margins = True)
    ost_raspr_USD.columns = get_multiindex(ost_raspr_USD.columns,'Сумма распределенного остатка,USD')
    ost_raspr_USD = ost_raspr_USD[:-1]
    
    gr = ['gr_index','ZKFGDOCIN']
    if columns_!=None:
        gr.extend(columns_)
    vl_06 = df[df['DOCTYPE'].isin(['VL06'])][gr].pivot_table(index=['gr_index'],aggfunc='sum',columns=columns_,values = ['ZKFGDOCIN'],fill_value=0,margins_name='Товар в пути(VL06),кол-во',margins = True)
    vl_06.columns = get_multiindex(vl_06.columns,'Товар в пути(VL06),кол-во')
    vl_06 = vl_06[:-1]
    
    
    znb_5 = df[df['DOCTYPE'].isin(['ZNB5'])][gr].pivot_table(index=['gr_index'],aggfunc='sum',columns=columns_,values = ['ZKFGDOCIN'],fill_value=0,margins_name='Товар в пути(ZNB5),кол-во',margins = True)
    znb_5.columns = get_multiindex(znb_5.columns,'Товар в пути(ZNB5),кол-во')
    znb_5 = znb_5[:-1]
    
    znb_2_3_4 = df[df['DOCTYPE'].isin(['ZNB2','ZNB3','ZNB4'])][gr].pivot_table(index=['gr_index'],aggfunc='sum',columns=columns_,values = ['ZKFGDOCIN'],fill_value=0,margins_name='Законтрактовано(ZNB2-4-ПртПзк),кол-во',margins = True)
    znb_2_3_4.columns = get_multiindex(znb_2_3_4.columns,'Законтрактовано(ZNB2-4-ПртПзк),кол-во')
    znb_2_3_4 = znb_2_3_4[:-1]
    
    nb = df[df['DOCTYPE'].apply(lambda x: len(re.findall('^NB\d\d',str(x)))>0 )][gr].pivot_table(index=['gr_index'],aggfunc='sum',columns=columns_,values = ['ZKFGDOCIN'],fill_value=0,margins_name='Заявка на закуп(NB),кол-во',margins = True)
    nb.columns = get_multiindex(nb.columns,'Заявка на закуп(NB),кол-во')
    nb = nb[:-1]
    
    
    gr = ['gr_index','ZKFGOUT']
    if columns_!=None:
        gr.extend(columns_)
    r_rash = df[gr].pivot_table(index=['gr_index'],aggfunc='sum',columns=columns_,values = ['ZKFGOUT'],fill_value=0,margins_name='Распределение расхода, кол-во',margins = True)
    r_rash.columns = get_multiindex(r_rash.columns,'Распределение расхода, кол-во')
    r_rash = r_rash[:-1]
    
    
    
    dfr = pd.concat([potr,potr_usd,ost_raspr,ost_raspr_KZT,ost_raspr_USD,vl_06,znb_5,znb_2_3_4,nb,r_rash],axis = 1)
    
    dfr = dfr.reset_index()
    
    for ind,col in enumerate(index_join_group):
        dfr[col] = dfr['index'].apply(lambda x: str(x).split('----')[ind])
    dfr = dfr.drop(['index'],axis = 1)
    
    for col in dfr.columns:
        if dfr[col].dtype=='float64':
            dfr[col].fillna(0,inplace = True)
            
#     if columns_==None:
        
# #         dfr.columns = [x[0] for x in dfr.columns]
#         print(dfr.columns)
    return dfr

In [ ]:
    tt1  = get_df_pvt_pandas(df = ar,index_ = ['COMP_CODE','/BIC/ZNPLANT'],columns_=['CALYEAR','/BIC/ZMODUL'])

In [ ]:
tt1

In [ ]:
tt1.to_excel(directory+'open_hub_.xlsx')

In [ ]:
ar = pd.concat([ar,ar,ar,ar,ar,ar,ar,ar,ar,ar,ar,ar,ar,ar,ar,ar,ar,ar,ar,ar,ar,ar,ar,ar,ar,ar,ar,ar,ar,ar,ar,ar,ar,ar,ar,ar,ar,ar,ar,ar,ar,ar,ar])

In [ ]:
potr_usd

In [ ]:
potr_usd.columns

In [ ]:
def get_multiindex(cols):
    arr = []
    for el in potr_usd.columns:
        a = list(el)
        a[0] = 'hrth'
        a = tuple(a)
        arr.append(a)
    return pd.MultiIndex.from_tuples(arr)


In [ ]:
pd.MultiIndex.from_tuples(arr)

In [ ]:
a  =['f','t']

In [ ]:
a[0] = 'ddd'

In [ ]:
a

In [ ]:
import copy
import csv
import dask
import pandas as pd
import re
import zipfile
import py7zr

def get_arr_from_csv(dir_and_filename):
    arr = []
    with open(dir_and_filename) as File:
        reader = csv.reader(File, delimiter=';', quotechar=',',
                            quoting=csv.QUOTE_MINIMAL)
        for row in reader:
            arr.append(row)
    
    return arr

def get_dask_arr(directory,file_data_name,file_meta_name,types_pnds,cols_all):
    arr_mtd = get_arr_from_csv(directory+file_meta_name+'.CSV')
    arr_data = get_arr_from_csv(directory+file_data_name+'.CSV')
    dict_meta = dict((x[0].strip(),x[1].strip()) for x in arr_mtd if len(x)>3 and x[0]!='COLUMN')
    dict_meta_types = dict((x[0].strip(),x[3].strip()) for x in arr_mtd if len(x)>3 and x[0]!='COLUMN')
    dfr = pd.DataFrame(data = arr_data,columns = list(dict_meta.values()))
    
    index_els = {}
    a = 1
    for el_0 in cols_all:
        for ind, el in  enumerate(dfr.columns):
            if el==el_0:
                index_els[str(a)] =ind+1 
                a+=1
    
        
    
    dfr = dfr[cols_all]
    for ind,col in enumerate(dfr.columns,1):
        
        if types_pnds[dict_meta_types[str(index_els[str(ind)])]]=='float64':
            
            dfr[col] = dfr[col].apply(lambda x:x[-1]+x[:-1] if len(re.findall(r'-$',x))>0 else x)
        dfr[col] = dfr[col].astype(types_pnds[dict_meta_types[str(index_els[str(ind)])]])
    
    sd = dfr
    return sd


def get_multiindex(cols,ins_value):
    arr = []
    if len(cols)>1:
        for el in cols:
            a = list(el)
            a[0] = ins_value
            a = tuple(a)
            arr.append(a)
        return pd.MultiIndex.from_tuples(arr)
    if len(cols)==1:
        return [ins_value]

def get_df_pvt_pandas(df,index_,columns_):
    index_join_group = copy.copy(index_) 

    vl06_index_join_group_start = copy.copy(index_join_group)
    vl06_index_join_group_start.append('ZKFGDOCIN')



#     print(index_join_group)
    
    df['gr_index']=''
    for ind, col in enumerate(index_join_group):
        if ind==len(index_join_group)-1:
            df['gr_index'] = df['gr_index'].astype('str')+df[col].astype('str')
        else:
            df['gr_index'] = df['gr_index'].astype('str')+df[col].astype('str')+'----'

    
    
    gr = ['gr_index','/BIC/ZMM1Q01']
    if columns_!=None:
        gr.extend(columns_)
    potr = df[gr].pivot_table(index=['gr_index'],aggfunc='sum',columns=columns_,values = ['/BIC/ZMM1Q01'],fill_value=0,margins_name='_Потребность,кол-во',margins = True)
    potr.columns = get_multiindex(potr.columns,'_Потребность,кол-во')
    potr = potr[:-1]
    
    
    gr = ['gr_index','/BIC/ZMM1A01']
    if columns_!=None:
        gr.extend(columns_)
    potr_usd = df[gr].pivot_table(index=['gr_index'],aggfunc='sum',columns=columns_,values = ['/BIC/ZMM1A01'],fill_value=0,margins_name='_Потребность,стоимость,USD',margins = True)
    potr_usd.columns = get_multiindex(potr_usd.columns,'_Потребность,стоимость,USD')
    potr_usd = potr_usd[:-1]
    
    
    gr = ['gr_index','/BIC/ZMM1Q22']
    if columns_!=None:
        gr.extend(columns_)
    ost_raspr = df[gr].pivot_table(index=['gr_index'],aggfunc='sum',columns=columns_,values = ['/BIC/ZMM1Q22'],fill_value=0,margins_name='Распределение остатка,кол-во',margins = True)
    ost_raspr.columns = get_multiindex(ost_raspr.columns,'Распределение остатка,кол-во')
    ost_raspr = ost_raspr[:-1]
    
    
    gr = ['gr_index','VALST_KZT']
    if columns_!=None:
        gr.extend(columns_)
    ost_raspr_KZT = df[gr].pivot_table(index=['gr_index'],aggfunc='sum',columns=columns_,values = ['VALST_KZT'],fill_value=0,margins_name='Сумма распределенного остатка,KZT',margins = True)
    ost_raspr_KZT.columns = get_multiindex(ost_raspr_KZT.columns,'Сумма распределенного остатка,KZT')
    ost_raspr_KZT = ost_raspr_KZT[:-1]
    
    gr = ['gr_index','VALST_USD']
    if columns_!=None:
        gr.extend(columns_)
    ost_raspr_USD = df[gr].pivot_table(index=['gr_index'],aggfunc='sum',columns=columns_,values = ['VALST_USD'],fill_value=0,margins_name='Сумма распределенного остатка,USD',margins = True)
    ost_raspr_USD.columns = get_multiindex(ost_raspr_USD.columns,'Сумма распределенного остатка,USD')
    ost_raspr_USD = ost_raspr_USD[:-1]
    
    gr = ['gr_index','ZKFGDOCIN']
    if columns_!=None:
        gr.extend(columns_)
    vl_06 = df[df['DOCTYPE'].isin(['VL06'])][gr].pivot_table(index=['gr_index'],aggfunc='sum',columns=columns_,values = ['ZKFGDOCIN'],fill_value=0,margins_name='Товар в пути(VL06),кол-во',margins = True)
    vl_06.columns = get_multiindex(vl_06.columns,'Товар в пути(VL06),кол-во')
    vl_06 = vl_06[:-1]
    
    
    znb_5 = df[df['DOCTYPE'].isin(['ZNB5'])][gr].pivot_table(index=['gr_index'],aggfunc='sum',columns=columns_,values = ['ZKFGDOCIN'],fill_value=0,margins_name='Товар в пути(ZNB5),кол-во',margins = True)
    znb_5.columns = get_multiindex(znb_5.columns,'Товар в пути(ZNB5),кол-во')
    znb_5 = znb_5[:-1]
    
    znb_2_3_4 = df[df['DOCTYPE'].isin(['ZNB2','ZNB3','ZNB4'])][gr].pivot_table(index=['gr_index'],aggfunc='sum',columns=columns_,values = ['ZKFGDOCIN'],fill_value=0,margins_name='Законтрактовано(ZNB2-4-ПртПзк),кол-во',margins = True)
    znb_2_3_4.columns = get_multiindex(znb_2_3_4.columns,'Законтрактовано(ZNB2-4-ПртПзк),кол-во')
    znb_2_3_4 = znb_2_3_4[:-1]
    
    nb = df[df['DOCTYPE'].apply(lambda x: len(re.findall('^NB\d\d',str(x)))>0 )][gr].pivot_table(index=['gr_index'],aggfunc='sum',columns=columns_,values = ['ZKFGDOCIN'],fill_value=0,margins_name='Заявка на закуп(NB),кол-во',margins = True)
    nb.columns = get_multiindex(nb.columns,'Заявка на закуп(NB),кол-во')
    nb = nb[:-1]
    
    
    gr = ['gr_index','ZKFGOUT']
    if columns_!=None:
        gr.extend(columns_)
    r_rash = df[gr].pivot_table(index=['gr_index'],aggfunc='sum',columns=columns_,values = ['ZKFGOUT'],fill_value=0,margins_name='Распределение расхода, кол-во',margins = True)
    r_rash.columns = get_multiindex(r_rash.columns,'Распределение расхода, кол-во')
    r_rash = r_rash[:-1]
    
    
    
    dfr = pd.concat([potr,potr_usd,ost_raspr,ost_raspr_KZT,ost_raspr_USD,vl_06,znb_5,znb_2_3_4,nb,r_rash],axis = 1)
    
    dfr = dfr.reset_index()
    
    for ind,col in enumerate(index_join_group):
        dfr[col] = dfr['index'].apply(lambda x: str(x).split('----')[ind])
    dfr = dfr.drop(['index'],axis = 1)
    
    for col in dfr.columns:
        if dfr[col].dtype=='float64':
            dfr[col].fillna(0,inplace = True)
            
#     if columns_==None:
        
# #         dfr.columns = [x[0] for x in dfr.columns]
#         print(dfr.columns)
    return dfr

# arr = []
# # directory = 'C:\\Users\\NKedrun\\AppData\\Local\\VirtualStore\\'
# directory = '\\\\kzukgs2sapf0.kazzinc.kz\\uz_reports_KIP\\test\\'
# types_pnds = {'CHAR':'category','CUKY':'category','CURR':'float64','FLTP':'float64','NUMC':'category','QUAN':'float64','UNIT':'category','DATS':'category'}

# ar = get_dask_arr(directory,'ZPR1OH01','S_ZPR1OH01')

# tt1  = get_df_pvt_pandas(df = ar,index_ = ['/BIC/ZNDOCTYPE'],columns_=None)

# tt2  = get_df_pvt_pandas(df = ar,index_ = ['CALYEAR'],columns_=None)


def get_report(years,months,directory_temp,directory_zip,index_,columns_):
    arr_cons = []
    types_pnds = {'CHAR':'category','CUKY':'category','CURR':'float64','FLTP':'float64','NUMC':'category','QUAN':'float64','UNIT':'category','DATS':'category'}
    cols_all = ['/BIC/ZMM1Q01','/BIC/ZMM1A01','/BIC/ZMM1Q22','VALST_KZT','VALST_USD','ZKFGDOCIN','DOCTYPE','ZKFGOUT']
    cols_all.extend(index_)
    if columns_ is not  None:
        cols_all.extend(columns_)
    
    for year in years:
        for month in months:
#             z = zipfile.ZipFile(directory_zip+str(year)+'\\'+str(month)+'\\VirtualStore.zip', 'r')
#             z.extractall(directory_temp)
            z = py7zr.SevenZipFile(directory_zip+str(year)+'\\'+str(month)+'\\VirtualStore.7z', mode='r')
            z.extractall(path=directory_temp)
            z.close()
            ar = get_dask_arr(directory_temp,'ZPR1OH01','S_ZPR1OH01',types_pnds,cols_all)
            arr_cons.append(ar)
            print('Загрузил '+str(year)+' '+str(month))
            
            
    cons = pd.concat(arr_cons)
    tt1  = get_df_pvt_pandas(df = cons,index_ = index_,columns_=columns_)
    
    return tt1

In [ ]:

directory_temp = 'C:\\Users\\NKedrun\\Desktop\\temp\\'
# directory_zip = 'C:\\Users\\NKedrun\\Desktop\\test\\'
directory_zip = '\\\\kzukgs2sapf0.kazzinc.kz\\uz_reports_KIP\\test\\'
index_ = ['MATERIAL','DATE0']
columns_ = ['/BIC/ZSTOCKCAT']
years = [2020]
months = [3,8,9]


df = get_report(years = years,months = months,index_ = index_,columns_ = columns_,directory_temp=directory_temp,directory_zip=directory_zip)

In [ ]:
df

In [ ]:
z.extractall('C:\\Users\\NKedrun\\Desktop\\temp\\')
z.close()

In [ ]:
user = 'NKEDRUN'
passwd = 'F292twmmwt292f!!!'
saprouter = 'kzukgs8bwop0.kazzinc.kz'

In [ ]:
conn = Connection(user=user, passwd=passwd,
                  mshost='CLient',
                  msserv='kzukgs8bwop0',
                  sysid='KIP',
                  group="PUBLIC",
                  saprouter=saprouter, 
                  client='500')

In [ ]:
from pyrfc import Connection
ASHOST='kzukgs8bwop1'
CLIENT='500'
SYSNR='KIP'
USER='NKEDRUN'
PASSWD='F292twmmwt292f!!!'
conn = Connection(ashost=ASHOST, sysnr=SYSNR, client=CLIENT, user=USER, passwd=PASSWD)

In [ ]:
b_result = conn.call('BAPI_USER_GET_DETAIL',
                     USERNAME = 'user',
                     CACHE_RESULTS  = ' ')

In [ ]:
options = [{ 'TEXT': "FCURR = 'USD'"}]
rowskips = 0
ROWS_AT_A_TIME = 10 

In [ ]:
result = conn.call('RFC_READ_TABLE',QUERY_TABLE = 'TCURR', OPTIONS = options, ROWSKIPS = rowskips, ROWCOUNT = ROWS_AT_A_TIME)

In [ ]:
conn.alive

In [ ]:
conn.call('ZPR1VPR29',QUERY_TABLE = 'F1 - Документы пополнения')

In [ ]:
conn.call('RFC_READ_TABLE',QUERY_TABLE = 'ZPR1VRP29_Q001')

In [ ]:
from pyrfc import Connection
ASHOST='kzukgs8bwboq0'
CLIENT='300'
SYSNR='KIQ'
USER='TEST_MMREPOR'
PASSWD='F292twmmwt292f!'
conn = Connection(ashost=ASHOST, sysnr=SYSNR, client=CLIENT, user=USER, passwd=PASSWD)

In [ ]:
conn.call('RFC_READ_TABLE',QUERY_TABLE = 'ZKAZ/ZKAZ_PR1/ZKAZ_PR1_VR/ZPR1VRP26/ZPR1VRP26_Q001')

In [ ]:
conn.call('BAPI_USER_GET_DETAIL',
                     USERNAME = 'TEST_MMREPOR',
                     CACHE_RESULTS  = ' ')

In [ ]:
from pyrfc import Connection, ABAPApplicationError, ABAPRuntimeError, LogonError, CommunicationError

from pprint import PrettyPrinter

def main():

    try:

        ASHOST='kzukgs8bwboq0'
        CLIENT='300'
        SYSNR='KIQ'
        USER='TEST_MMREPOR'
        PASSWD='F292twmmwt292f!'
        conn = Connection(ashost=ASHOST, sysnr=SYSNR, client=CLIENT, user=USER, passwd=PASSWD)

        options = [{ 'TEXT': "FCURR = 'USD'"}]
        pp = PrettyPrinter(indent=4)
        ROWS_AT_A_TIME = 10 
        rowskips = 0

        while True:
            print ("----Begin of Batch---")
            result = conn.call('RFC_READ_TABLE', \
                                QUERY_TABLE = 'TCURR', \
                                OPTIONS = options, \
                                ROWSKIPS = rowskips, ROWCOUNT = ROWS_AT_A_TIME)
            pp.pprint(result['DATA'])
            rowskips += ROWS_AT_A_TIME
            if len(result['DATA']) < ROWS_AT_A_TIME:
                break

    except CommunicationError:
        print ("Could not connect to server.")
        raise
    except LogonError:
        print ("Could not log in. Wrong credentials?")
        raise
    except (ABAPApplicationError, ABAPRuntimeError):
        print ("An error occurred.")
        raise

In [ ]:
from pyrfc import Connection
ASHOST='kzukgs8bwboq0'
CLIENT='300'
SYSNR='KIQ'
USER='NKEDRUN'
PASSWD='F292twmmwt292f!!!'
conn = Connection(ashost=ASHOST, sysnr=SYSNR, client=CLIENT, user=USER, passwd=PASSWD)

In [ ]:

result = conn.call('RFC_READ_TABLE',QUERY_TABLE = "/BIC/AZPR1DPA522",DELIMITER='|')

In [ ]:
result['FIELDS']

In [ ]:
[x for x  in result['DATA'][0]['WA'].split('|')]

In [ ]:
rowskips = 0
ROWS_AT_A_TIME = 1000
fiels = ['CALYEAR','ZKFGOUT']
result = conn.call('RFC_READ_TABLE',QUERY_TABLE = "/BIC/AZPR1DPA572",FIELDS = fiels,ROWSKIPS = rowskips, ROWCOUNT = ROWS_AT_A_TIME,DELIMITER='|')

In [ ]:
lst = []

In [ ]:
lst = []
rowskips = 0
ROWS_AT_A_TIME = 300000
fiels = ['CALYEAR','PLANT','VENDOR','MATERIAL']
options = [{ 'TEXT': "Z0DATE_MSMR_01 = 2020/10/1"}]
while True:
    result = conn.call('RFC_READ_TABLE',\
                       QUERY_TABLE = "/BIC/AZPR1DPA542",\
                       FIELDS = fiels,\
                       ROWSKIPS = rowskips, \
                       ROWCOUNT = ROWS_AT_A_TIME,DELIMITER='|',OPTIONS = options)
    
    for x in [x['WA'].split('|') for x  in  result['DATA']]:
        lst.append(x)
        
    rowskips += ROWS_AT_A_TIME  
    
    print(str(len(lst))+'_'+str(rowskips))     
    if len(result['DATA']) < ROWS_AT_A_TIME:
        break

In [ ]:
pd.DataFrame(lst)

In [ ]:
pd.DataFrame()

In [ ]:
import py7zr
archive = py7zr.SevenZipFile('C:\\Users\\NKedrun\\Desktop\\temp\\temp.7z', mode='r')
archive.extractall(path="C:\\Users\\NKedrun\\Desktop\\test\\")

In [ ]:

def get_arr_from_csv(dir_and_filename):
    arr = []
    with open(dir_and_filename,encoding='utf-8') as File:
        reader = csv.reader(File, delimiter=';', quotechar=',',
                            quoting=csv.QUOTE_MINIMAL)
        for row in reader:
            arr.append(row)
    
    return arr

def get_dask_arr(directory,file_data_name,file_meta_name,types_pnds,cols_all):
    arr_mtd = get_arr_from_csv(directory+file_meta_name+'.CSV')
    arr_data = get_arr_from_csv(directory+file_data_name+'.CSV')
    dict_meta = dict((x[0].strip(),x[1].strip()) for x in arr_mtd if len(x)>3 and x[0]!='COLUMN')
    dict_meta_types = dict((x[0].strip(),x[3].strip()) for x in arr_mtd if len(x)>3 and x[0]!='COLUMN')
    dfr = pd.DataFrame(data = arr_data,columns = list(dict_meta.values()))
    
    index_els = {}
    a = 1
    for el_0 in cols_all:
        for ind, el in  enumerate(dfr.columns):
            if el==el_0:
                index_els[str(a)] =ind+1 
                a+=1
    
        
    
    dfr = dfr[cols_all]
    for ind,col in enumerate(dfr.columns,1):
        
        if types_pnds[dict_meta_types[str(index_els[str(ind)])]]=='float64':
            
            dfr[col] = dfr[col].apply(lambda x:x[-1]+x[:-1] if len(re.findall(r'-$',x))>0 else x)
        dfr[col] = dfr[col].astype(types_pnds[dict_meta_types[str(index_els[str(ind)])]])
    
    sd = dfr
    return sd




def get_report(years,months,directory_temp,directory_zip,index_,columns_):
    arr_cons = []
    types_pnds = {'CHAR':'category','CUKY':'category','CURR':'float64','FLTP':'float64','NUMC':'category','QUAN':'float64','UNIT':'category','DATS':'category'}
    cols_all = ['/BIC/ZMM1Q01','/BIC/ZMM1A01','/BIC/ZMM1Q22','VALST_KZT','VALST_USD','ZKFGDOCIN','DOCTYPE','ZKFGOUT']
    cols_all.extend(index_)
    if columns_ is not  None:
        cols_all.extend(columns_)
    
  
    ar = get_dask_arr(directory_temp,'F1_EXTRACTION_2021_04_07','S_F1_EXTRACTION_2021_04_07',types_pnds,cols_all)
    arr_cons.append(ar)

            
            
    cons = pd.concat(arr_cons)
    return cons
#     tt1  = get_df_pvt_pandas(df = cons,index_ = index_,columns_=columns_)

In [ ]:
directory_temp = 'C:\\Users\\NKedrun\\Desktop\\openHub_arr\\'
# directory_zip = 'C:\\Users\\NKedrun\\Desktop\\test\\'
directory_zip = 'C:\\Users\\NKedrun\\Desktop\\VirtualStore\\'
index_ = ['MATERIAL','DATE0']
columns_ = ['/BIC/ZSTOCKCAT']
years = [2020]
months = [3,8,9]


df = get_report(years = years,months = months,index_ = index_,columns_ = columns_,directory_temp=directory_temp,directory_zip=directory_zip)

In [ ]:
df['DATE0'].unique()

In [ ]:
df

In [ ]:
tt1  = get_df_pvt_pandas(df = df,index_ = index_,columns_=columns_)

In [ ]:
tt12 = tt1[tt1['DATE0']=='20210218']

In [ ]:
tt12.columns

In [ ]:
tt12['Сумма распределенного остатка,KZT']['Сумма распределенного остатка,KZT'].sum()

In [ ]:
df[df['DATE0']=='00000000']

In [ ]:
df

In [ ]:
import pandas as pd 
from tqdm import tqdm


In [ ]:
PATH = 'C:\\Users\\NKedrun\\Desktop\\openHub_arr\\F1_EXTRACTION_2021_04_07.csv'
chunksize = 500000 

cols = list(traintypes.keys())

df_list = [] # list to hold the batch dataframe

for df_chunk in tqdm(pd.read_csv(PATH, chunksize=chunksize,delimiter=';', quotechar=',')):
    # Can process each chunk of dataframe here
    # clean_data(), feature_engineer(),fit()

    # Alternatively, append the chunk to list and merge all
    df_list.append(df_chunk) 

# Merge all dataframes into one dataframe
X = pd.concat(df_list)

# Delete the dataframe list to release memory
del df_list
del df_chunk

In [ ]:
df_list

In [ ]:
X

In [ ]:
import pandas as pd
import datetime
pd.set_option('display.max_columns', None)
from pyexcelerate import Workbook
from decimal import Decimal
import numpy as np
import time
import re
from tqdm import tqdm
import warnings
import csv
import copy
warnings.filterwarnings('ignore')

def get_multiindex(cols,ins_value):
    arr = []
    if len(cols)>1:
        for el in cols:
            a = list(el)
            a[0] = ins_value
            a = tuple(a)
            arr.append(a)
        return pd.MultiIndex.from_tuples(arr)
    if len(cols)==1:
        return [ins_value]


def get_df_pvt_pandas(df,index_,columns_,filters_):
    index_join_group = copy.copy(index_) 

    vl06_index_join_group_start = copy.copy(index_join_group)
    vl06_index_join_group_start.append('ZKFGDOCIN')
    
    getTrFalse = df.isin(filters_)
    arr =  np.array(getTrFalse)
    index_ = getTrFalse[np.sum(arr,axis = 1)==len(filters.keys())].index
    df = df.loc[index_]
    if len(df)==0:
        print('Данных по данному фильтру нет')
        return 
#     df = df[df[list(filters_.keys())[0]].isin(filters_[list(filters_.keys())[0]])]

#     print(index_join_group)
    
    df['gr_index']=''
    for ind, col in enumerate(index_join_group):
        if ind==len(index_join_group)-1:
            df['gr_index'] = df['gr_index'].astype('str')+df[col].astype('str')
        else:
            df['gr_index'] = df['gr_index'].astype('str')+df[col].astype('str')+'----'

    
    
    gr = ['gr_index','/BIC/ZMM1Q01']
    if columns_!=None:
        gr.extend(columns_)
    potr = df[gr].pivot_table(index=['gr_index'],aggfunc='sum',columns=columns_,values = ['/BIC/ZMM1Q01'],fill_value=0,margins_name='_Потребность,кол-во',margins = True)
    potr.columns = get_multiindex(potr.columns,'_Потребность,кол-во')
    potr = potr[:-1]
    
    
    gr = ['gr_index','/BIC/ZMM1A01']
    if columns_!=None:
        gr.extend(columns_)
    potr_usd = df[gr].pivot_table(index=['gr_index'],aggfunc='sum',columns=columns_,values = ['/BIC/ZMM1A01'],fill_value=0,margins_name='_Потребность,стоимость,USD',margins = True)
    potr_usd.columns = get_multiindex(potr_usd.columns,'_Потребность,стоимость,USD')
    potr_usd = potr_usd[:-1]
    
    
    gr = ['gr_index','/BIC/ZMM1Q22']
    if columns_!=None:
        gr.extend(columns_)
    ost_raspr = df[gr].pivot_table(index=['gr_index'],aggfunc='sum',columns=columns_,values = ['/BIC/ZMM1Q22'],fill_value=0,margins_name='Распределение остатка,кол-во',margins = True)
    ost_raspr.columns = get_multiindex(ost_raspr.columns,'Распределение остатка,кол-во')
    ost_raspr = ost_raspr[:-1]
    
    
    gr = ['gr_index','VALST_KZT']
    if columns_!=None:
        gr.extend(columns_)
    ost_raspr_KZT = df[gr].pivot_table(index=['gr_index'],aggfunc='sum',columns=columns_,values = ['VALST_KZT'],fill_value=0,margins_name='Сумма распределенного остатка,KZT',margins = True)
    ost_raspr_KZT.columns = get_multiindex(ost_raspr_KZT.columns,'Сумма распределенного остатка,KZT')
    ost_raspr_KZT = ost_raspr_KZT[:-1]
    
    gr = ['gr_index','VALST_USD']
    if columns_!=None:
        gr.extend(columns_)
    ost_raspr_USD = df[gr].pivot_table(index=['gr_index'],aggfunc='sum',columns=columns_,values = ['VALST_USD'],fill_value=0,margins_name='Сумма распределенного остатка,USD',margins = True)
    ost_raspr_USD.columns = get_multiindex(ost_raspr_USD.columns,'Сумма распределенного остатка,USD')
    ost_raspr_USD = ost_raspr_USD[:-1]
    
    gr = ['gr_index','ZKFGDOCIN']
    if columns_!=None:
        gr.extend(columns_)
        
        
    if df[df['DOCTYPE'].isin(['VL06'])].shape[0]>0:    
        vl_06 = df[df['DOCTYPE'].isin(['VL06'])][gr].pivot_table(index=['gr_index'],aggfunc='sum',columns=columns_,values = ['ZKFGDOCIN'],fill_value=0,margins_name='Товар в пути(VL06),кол-во',margins = True)
        vl_06.columns = get_multiindex(vl_06.columns,'Товар в пути(VL06),кол-во')        
        vl_06 = vl_06[:-1]
    else:
        vl_06 = pd.DataFrame(columns=[('Товар в пути(VL06),кол-во', 'Данных нет')])
        vl_06.columns = pd.MultiIndex.from_tuples([('Товар в пути(VL06),кол-во', 'Данных нет')]) 
        vl_06 = vl_06[:-1]
            
    
    if df[df['DOCTYPE'].isin(['ZNB5'])].shape[0]>0:
        znb_5 = df[df['DOCTYPE'].isin(['ZNB5'])][gr].pivot_table(index=['gr_index'],aggfunc='sum',columns=columns_,values = ['ZKFGDOCIN'],fill_value=0,margins_name='Товар в пути(ZNB5),кол-во',margins = True)
        znb_5.columns = get_multiindex(znb_5.columns,'Товар в пути(ZNB5),кол-во')
        znb_5 = znb_5[:-1]
    else:

        znb_5 = pd.DataFrame(columns=[('Товар в пути(ZNB5),кол-во', 'Данных нет')])
        znb_5.columns = pd.MultiIndex.from_tuples([('Товар в пути(ZNB5),кол-во', 'Данных нет')]) 
        znb_5 = znb_5[:-1]
    
    
    if df[df['DOCTYPE'].isin(['ZNB2','ZNB3','ZNB4'])].shape[0]>0:
        znb_2_3_4 = df[df['DOCTYPE'].isin(['ZNB2','ZNB3','ZNB4'])][gr].pivot_table(index=['gr_index'],aggfunc='sum',columns=columns_,values = ['ZKFGDOCIN'],fill_value=0,margins_name='Законтрактовано(ZNB2-4-ПртПзк),кол-во',margins = True)
        znb_2_3_4.columns = get_multiindex(znb_2_3_4.columns,'Законтрактовано(ZNB2-4-ПртПзк),кол-во')
        znb_2_3_4 = znb_2_3_4[:-1]
    else:
        znb_2_3_4 = pd.DataFrame(columns=[('Законтрактовано(ZNB2-4-ПртПзк),кол-во', 'Данных нет')])
        znb_2_3_4.columns = pd.MultiIndex.from_tuples([('Законтрактовано(ZNB2-4-ПртПзк),кол-во', 'Данных нет')]) 
        znb_2_3_4 = znb_2_3_4[:-1]
    
    nb = df[df['DOCTYPE'].apply(lambda x: len(re.findall('^NB\d\d',str(x)))>0 )][gr].pivot_table(index=['gr_index'],aggfunc='sum',columns=columns_,values = ['ZKFGDOCIN'],fill_value=0,margins_name='Заявка на закуп(NB),кол-во',margins = True)
    nb.columns = get_multiindex(nb.columns,'Заявка на закуп(NB),кол-во')
    nb = nb[:-1]
    
    
    gr = ['gr_index','ZKFGOUT']
    if columns_!=None:
        gr.extend(columns_)
    r_rash = df[gr].pivot_table(index=['gr_index'],aggfunc='sum',columns=columns_,values = ['ZKFGOUT'],fill_value=0,margins_name='Распределение расхода, кол-во',margins = True)
    r_rash.columns = get_multiindex(r_rash.columns,'Распределение расхода, кол-во')
    r_rash = r_rash[:-1]
    
    
    
    
    gr = ['gr_index','ZKFGIN']
    if columns_!=None:
        gr.extend(columns_)
        
    if df[df['/BIC/ZDECADE'].isin([1])].shape[0]>0:    
        r_prih_1 = df[df['/BIC/ZDECADE'].isin([1])][gr].pivot_table(index=['gr_index'],aggfunc='sum',columns=columns_,values = ['ZKFGIN'],fill_value=0,margins_name='Распределение прихода 1декада, кол-во',margins = True)
        r_prih_1.columns = get_multiindex(r_prih_1.columns,'Распределение прихода 1декада, кол-во')
        r_prih_1 = r_prih_1[:-1]
    else:
        r_prih_1 = pd.DataFrame(columns=[('Распределение прихода 1декада, кол-во', 'Данных нет')])
        r_prih_1.columns = pd.MultiIndex.from_tuples([('Распределение прихода 1декада, кол-во', 'Данных нет')]) 
        r_prih_1 = r_prih_1[:-1]
        
        
    
    if df[df['/BIC/ZDECADE'].isin([2])].shape[0]>0:    
        r_prih_2 = df[df['/BIC/ZDECADE'].isin([2])][gr].pivot_table(index=['gr_index'],aggfunc='sum',columns=columns_,values = ['ZKFGIN'],fill_value=0,margins_name='Распределение прихода 2декада, кол-во',margins = True)
        r_prih_2.columns = get_multiindex(r_prih_2.columns,'Распределение прихода 2декада, кол-во')
        r_prih_2 = r_prih_2[:-1]
    else:
        r_prih_2 = pd.DataFrame(columns=[('Распределение прихода 2декада, кол-во', 'Данных нет')])
        r_prih_2.columns = pd.MultiIndex.from_tuples([('Распределение прихода 2декада, кол-во', 'Данных нет')]) 
        r_prih_2 = r_prih_2[:-1]
        
        
        
    if df[df['/BIC/ZDECADE'].isin([3])].shape[0]>0:    
        r_prih_3 = df[df['/BIC/ZDECADE'].isin([3])][gr].pivot_table(index=['gr_index'],aggfunc='sum',columns=columns_,values = ['ZKFGIN'],fill_value=0,margins_name='Распределение прихода 3декада, кол-во',margins = True)
        r_prih_3.columns = get_multiindex(r_prih_2.columns,'Распределение прихода 3декада, кол-во')
        r_prih_3 = r_prih_3[:-1]
    else:
        r_prih_3 = pd.DataFrame(columns=[('Распределение прихода 3декада, кол-во', 'Данных нет')])
        r_prih_3.columns = pd.MultiIndex.from_tuples([('Распределение прихода 3декада, кол-во', 'Данных нет')]) 
        r_prih_3 = r_prih_3[:-1]
    
    
    
    gr = ['gr_index','/BIC/ZCPPVCC']
    if columns_!=None:
        gr.extend(columns_)
    
    
    
    if df[df['FLAG_DISTR'].isin(['I','N','P','T'])].shape[0]>0:    
        r_prih_sum = df[df['FLAG_DISTR'].isin(['I','N','P','T'])][gr].pivot_table(index=['gr_index'],aggfunc='sum',columns=columns_,values = ['/BIC/ZCPPVCC'],fill_value=0,margins_name='Сумма распределенного прихода, USD',margins = True)
        r_prih_sum.columns = get_multiindex(r_prih_sum.columns,'Сумма распределенного прихода, USD')
        r_prih_sum = r_prih_sum[:-1]
    else:
        r_prih_sum = pd.DataFrame(columns=[('Сумма распределенного прихода, USD', 'Данных нет')])
        r_prih_sum.columns = pd.MultiIndex.from_tuples([('Сумма распределенного прихода, USD', 'Данных нет')]) 
        r_prih_sum = r_prih_sum[:-1]
        
        
    if df[df['FLAG_DISTR'].isin(['U','O'])].shape[0]>0:    
        r_rash_sum = df[df['FLAG_DISTR'].isin(['U','O'])][gr].pivot_table(index=['gr_index'],aggfunc='sum',columns=columns_,values = ['/BIC/ZCPPVCC'],fill_value=0,margins_name='Сумма распределенного расхода, USD',margins = True)
        r_rash_sum.columns = get_multiindex(r_rash_sum.columns,'Сумма распределенного расхода, USD')
        r_rash_sum = r_rash_sum[:-1]
    else:
        r_rash_sum = pd.DataFrame(columns=[('Сумма распределенного расхода, USD', 'Данных нет')])
        r_rash_sum.columns = pd.MultiIndex.from_tuples([('Сумма распределенного расхода, USD', 'Данных нет')]) 
        r_rash_sum = r_rash_sum[:-1]

        
    gr = ['gr_index','CPPVLC']
    if columns_!=None:
        gr.extend(columns_)
        
        
    if df[df['FLAG_DISTR'].isin(['I','N','P','T'])].shape[0]>0:    
        r_prih_sum_KZT = df[df['FLAG_DISTR'].isin(['I','N','P','T'])][gr].pivot_table(index=['gr_index'],aggfunc='sum',columns=columns_,values = ['CPPVLC'],fill_value=0,margins_name='Сумма распределенного прихода, KZT',margins = True)
        r_prih_sum_KZT.columns = get_multiindex(r_prih_sum_KZT.columns,'Сумма распределенного прихода, KZT')
        r_prih_sum_KZT = r_prih_sum_KZT[:-1]
#         print(r_prih_sum_KZT)
    else:
        r_prih_sum_KZT = pd.DataFrame(columns=[('Сумма распределенного прихода, KZT', 'Данных нет')])
        r_prih_sum_KZT.columns = pd.MultiIndex.from_tuples([('Сумма распределенного прихода, KZT', 'Данных нет')]) 
        r_prih_sum_KZT = r_prih_sum_KZT[:-1]
        
        
        
    if df[df['FLAG_DISTR'].isin(['U','O'])].shape[0]>0:    
        r_rash_sum_KZT = df[df['FLAG_DISTR'].isin(['U','O'])][gr].pivot_table(index=['gr_index'],aggfunc='sum',columns=columns_,values = ['CPPVLC'],fill_value=0,margins_name='Сумма распределенного расхода, KZT',margins = True)
        r_rash_sum_KZT.columns = get_multiindex(r_rash_sum_KZT.columns,'Сумма распределенного расхода, KZT')
        r_rash_sum_KZT = r_rash_sum_KZT[:-1]
    else:
        r_rash_sum_KZT = pd.DataFrame(columns=[('Сумма распределенного расхода, KZT', 'Данных нет')])
        r_rash_sum_KZT.columns = pd.MultiIndex.from_tuples([('Сумма распределенного расхода, KZT', 'Данных нет')]) 
        r_rash_sum_KZT = r_rash_sum_KZT[:-1]

    
   

    
    
    
    
    dfr = pd.concat([potr,potr_usd,ost_raspr,ost_raspr_KZT,ost_raspr_USD,vl_06,znb_5,znb_2_3_4,nb,r_rash,r_rash_sum,r_rash_sum_KZT,r_prih_1,r_prih_2,r_prih_3,r_prih_sum,r_prih_sum_KZT],axis = 1)
    
    dfr = dfr.reset_index()
    
    for ind,col in enumerate(index_join_group):
        dfr[col] = dfr['index'].apply(lambda x: str(x).split('----')[ind])
    dfr = dfr.drop(['index'],axis = 1)
    
    
    for col in dfr.columns:
#         print(type(dfr[col]))
        if dfr[col].dtype=='float64':
            dfr[col].fillna(0,inplace = True)
            
#     if columns_==None:
        
# #         dfr.columns = [x[0] for x in dfr.columns]
#         print(dfr.columns)
    return dfr

# arr = []
# # directory = 'C:\\Users\\NKedrun\\AppData\\Local\\VirtualStore\\'
# directory = '\\\\kzukgs2sapf0.kazzinc.kz\\uz_reports_KIP\\test\\'
# types_pnds = {'CHAR':'category','CUKY':'category','CURR':'float64','FLTP':'float64','NUMC':'category','QUAN':'float64','UNIT':'category','DATS':'category'}

# ar = get_dask_arr(directory,'ZPR1OH01','S_ZPR1OH01')

# tt1  = get_df_pvt_pandas(df = ar,index_ = ['/BIC/ZNDOCTYPE'],columns_=None)

# tt2  = get_df_pvt_pandas(df = ar,index_ = ['CALYEAR'],columns_=None)





def get_arr_from_csv(dir_and_filename):
    arr = []
    with open(dir_and_filename,encoding='utf-8') as File:
        reader = csv.reader(File, delimiter=';', quotechar=',',
                            quoting=csv.QUOTE_MINIMAL)
        for row in reader:
            arr.append(row)
    
    return arr


def get_dask_arr(directory,file_data_name,file_meta_name,types_pnds,cols_all):
    arr_mtd = get_arr_from_csv(directory+file_meta_name+'.CSV')
#     arr_data = get_arr_from_csv(directory+file_data_name+'.CSV')
    dict_meta = dict((x[0].strip(),x[1].strip()) for x in arr_mtd if len(x)>3 and x[0]!='COLUMN')
    dict_meta_types = dict((x[0].strip(),x[3].strip()) for x in arr_mtd if len(x)>3 and x[0]!='COLUMN')
    
    chunksize = 1000000 


    df_list = [] # list to hold the batch dataframe

    for df_chunk in tqdm(pd.read_csv(directory+file_data_name+'.CSV',quoting=csv.QUOTE_MINIMAL,header=None, chunksize=chunksize,delimiter=';', quotechar=',')):
        # Can process each chunk of dataframe here
        # clean_data(), feature_engineer(),fit()

        # Alternatively, append the chunk to list and merge all
        df_list.append(df_chunk) 

    # Merge all dataframes into one dataframe
    dfr = pd.concat(df_list)
    dfr.columns = list(dict_meta.values())
    # Delete the dataframe list to release memory
    del df_list
    del df_chunk
    
    print('Загрузили,начинаем обработку')
    
    index_els = {}
    a = 1
    for el_0 in cols_all:
        for ind, el in  enumerate(dfr.columns):
            if el==el_0:
                index_els[str(a)] =ind+1 
                a+=1
    
        
    
    dfr = dfr[cols_all]
    for ind,col in enumerate(dfr.columns,1):
        
        if types_pnds[dict_meta_types[str(index_els[str(ind)])]]=='float64':
            
            dfr[col] = dfr[col].astype('str').apply(lambda x:x[-1]+x[:-1] if len(re.findall(r'-$',x))>0 else x)
        dfr[col].fillna('',inplace = True)
        dfr[col] = dfr[col].astype(types_pnds[dict_meta_types[str(index_els[str(ind)])]])
    
    sd = dfr
#     sd = sd.fillna(0)
    print('Загрузка данных завершена')
    return sd


def get_report(directory_temp,index_,columns_,filename,filename_meta):
    arr_cons = []
    types_pnds = {'CHAR':'category','CUKY':'category','CURR':'float64','FLTP':'float64','NUMC':'category','QUAN':'float64','UNIT':'category','DATS':'category'}
    cols_all = ['/BIC/ZMM1Q01','/BIC/ZMM1A01','/BIC/ZMM1Q22','VALST_KZT','VALST_USD','ZKFGDOCIN','DOCTYPE','ZKFGOUT','/BIC/ZDECADE','FLAG_DISTR','/BIC/ZCPPVCC','ZKFGIN','CPPVLC']
    cols_all.extend(index_)
    if columns_ is not  None:
        cols_all.extend(columns_)
    
  
    ar = get_dask_arr(directory_temp,filename,filename_meta,types_pnds,cols_all)
#     arr_cons.append(ar)

            
            
#     cons = pd.concat(arr_cons)
    return ar
#     tt1  = get_df_pvt_pandas(df = cons,index_ = index_,columns_=columns_)

In [ ]:
filename = 'F1_EXTRACTION_2021_04_07'
filename_meta = 'S_F1_EXTRACTION_2021_04_07'

directory_temp = 'C:\\Users\\NKedrun\\Desktop\\openHub_arr\\' # Не менять
directory_report = 'C:\\Users\\NKedrun\\Desktop\\' # Не менять Директория в которой появится файл отчета

columns_ = ['MATERIAL','/BIC/ZMODUL','/BIC/ZSTCKTGT','/BIC/ZCOMPLEKS','/BIC/ZNSTOR','/BIC/ZBATCHMOD','COMP_CODE'] # Добавлять интересующе стлбцы



df = get_report(index_ = ['DATE0'] ,columns_ = columns_,directory_temp=directory_temp,filename = filename,filename_meta = filename_meta)

In [ ]:

filters = {'MATERIAL':['911201188'],
           'COMP_CODE':['3120']} # фильтр для формирования отчета



index_cons = ['MATERIAL','DATE0','/BIC/ZSTCKTGT','COMP_CODE']
report  = get_df_pvt_pandas(df = df,index_ = index_cons,columns_=None,filters_ = filters)


report.to_excel(directory_report+'open_hub_upload.xlsx')

In [ ]:
tt1

In [ ]:
tt1

In [ ]:
df['COMP_CODE']

In [ ]:
getTrFalse = df.isin(filters)
dd = df[getTrFalse]


In [ ]:
df.loc[dd.dropna(axis='index', how='all').index]

In [ ]:
getTrFalse[getTrFalse['MATERIAL'] & getTrFalse['COMP_CODE'] ]

In [ ]:
ind_t = []

for ind_ in range(len(arr)):
    s = sum(arr[ind_])
    if s==2:
        ind_t.append(ind_)

In [ ]:
getTrFalse[]

In [ ]:
filters = {'MATERIAL':['010101012'],'COMP_CODE':['3120']}

In [ ]:
dd

In [ ]:
arr = np.array(getTrFalse)

In [ ]:
np.sum(arr,axis = 1)

In [ ]:
arr =  np.array(getTrFalse)
getTrFalse[np.sum(arr,axis = 1)==len(filters.keys())].index

In [ ]:
len(filters.keys())

In [ ]:
len(df)

In [ ]:
assert len(df)==0,'Привет'